In [2]:
import surprise

print(surprise.__version__)

1.1.1


### Recommendation Algorithm using Surprise

In [4]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
# load data and seperate into train and test set
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\DooDoo/.surprise_data/ml-100k


In [6]:
# use SVD to train
algo = SVD()
algo.fit(trainset)

In [7]:
# predict ratings use .test() method
predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:', len(predictions))
predictions[:5]

prediction type: <class 'list'> size: 25000


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.56338711038062, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.863980790807576, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.027094326009809, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6568390582537975, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.434685371764225, details={'was_impossible': False})]

In [8]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.56338711038062),
 ('882', '291', 3.863980790807576),
 ('535', '507', 4.027094326009809)]

In [9]:
# predict() is for individual user, and item
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.43   {'was_impossible': False}


In [10]:
# get rmse prediction
accuracy.rmse(predictions)

RMSE: 0.9454


0.9453529779579908

### Load ratings data from before

In [11]:
import pandas as pd
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
# save without header
ratings.to_csv('./ml-latest-small/rating_noh.csv', index = False, header=False)

In [14]:
# use Reader to load file for surprise
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./ml-latest-small/rating_noh.csv', reader=reader)

In [15]:
# evaluate using SVD
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

# train, test, get rmse
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [16]:
# using pandas Dataframe
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Cross Validation and hyper parameter tuning

In [18]:
# using cross_validate
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8719  0.8699  0.8793  0.8710  0.8727  0.8730  0.0033  
MAE (testset)     0.6702  0.6696  0.6773  0.6686  0.6701  0.6712  0.0031  
Fit time          5.22    5.65    5.20    5.05    5.76    5.38    0.28    
Test time         0.15    0.21    0.20    0.12    0.14    0.16    0.03    


{'test_rmse': array([0.8719282 , 0.86994999, 0.87925444, 0.87102874, 0.87265233]),
 'test_mae': array([0.67021433, 0.66961937, 0.67726117, 0.66862448, 0.67007689]),
 'fit_time': (5.215017318725586,
  5.650851726531982,
  5.196099042892456,
  5.053480863571167,
  5.762583494186401),
 'test_time': (0.15259289741516113,
  0.20545530319213867,
  0.20046448707580566,
  0.11971354484558105,
  0.13863158226013184)}

In [19]:
#using GridSearchCV
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors':[50, 100, 200]}

# cv = 3 folds, put in the plain class name, instead of pre-declared class
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8761213856340652
{'n_epochs': 20, 'n_factors': 50}


### Use DatasetAutoFold to get trainset 

In [22]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/rating_noh.csv', reader=reader)

# get full data as trainset
trainset = data_folds.build_full_trainset()

In [23]:
# use SVD to train
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [25]:
# get DataFrame movies to get movie data
movies = pd.read_csv('./ml-latest-small/movies.csv')

#get userId=9 data and check if user rated movieId=42
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('UserId 9 have not rated movie 42 yet')
    
print(movies[movies['movieId']==42])

UserId 9 have not rated movie 42 yet
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [26]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [27]:
# get the number of movies rated, recommended and total

def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']==9]['movieId'].tolist()
    
    total_movies = movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print("number of rated movies:", len(seen_movies), 'number of recommended movies:', len(unseen_movies), \
         'number of all movies', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

number of rated movies: 46 number of recommended movies: 9696 number of all movies 9742


In [28]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # use predict method for all the unseen movies
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # prediction list has predictions objects where
    # [Prediction(uid='9', iid='1', est=3.69), ...]
    # to sort by est values, we need a sortkey_est function
    def sortkey_est(pred):
        return pred.est
    
    # usr sortkey_est() to sort and get top_n
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions=predictions[:top_n]
    
    #get movieId, estimated rating, and movie names
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('### Top-10 recommended movie list ###')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

number of rated movies: 46 number of recommended movies: 9696 number of all movies 9742
### Top-10 recommended movie list ###
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
